In [1]:
# Import global libraries
import numpy as np
import matplotlib.pyplot as plt
from time import time
from PIL import Image
from tifffile import imread, imsave
from importlib import reload

# Import local libraries
import stackregistration

%matplotlib

/Users/Ben/Code/anaconda2/envs/py3/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Using matplotlib backend: Qt4Agg


In [3]:
# Load data
f="../../../sample_data/tiffs/50_s40_3.6mx_300kV_0.5us_1024px_1.tif"
stack=np.rollaxis(imread(f),0,3)
stack=stack[:,:,:]/float(2**16)

In [4]:
# Examine data
for i in range(4):
    fig,ax=plt.subplots()
    ax.matshow(stack[:,:,i],cmap='gray')
    plt.show()

In [5]:
# Instantiate imstack object
s=stackregistration.imstack(stack[:,:,:10])

In [6]:
# Get all FFTs
s.getFFTs()

In [7]:
# Make Fourier mask
s.makeFourierMask(mask="lowpass",n=4)
s.show_Fourier_mask()

In [10]:
# Change parameters for Gaussian fits and/or CoM detection
s.setGaussianFitParams(num_peaks=3,sigma_guess=2,window_radius=6)

In [13]:
# Find all image shifts
t0=time()
s.findImageShifts(correlationType='cc',findMaxima='gf')
t=time()-t0
print("Performed {} correlations in {} minutes {} seconds".format(s.nz*(s.nz-1)/2,int(t/60),t%60))

Correlating images 0 and 1


NameError: name 'cc_shifts' is not defined

In [12]:
# Show Xij and Yij matrices
s.show_Rij()

In [37]:
# Create mask, defining unuseable data points
s.set_nz(0,20)               # Set min/maz useable image indices
s.set_bad_images([])         # Set bad images
s.get_outliers("NN",9)       # Set outlier threshhold
s.show_Rij(mask=True)

In [38]:
# Create registered image stack and average
s.get_averaged_image()

In [39]:
# Display
s.show()

In [26]:
# Show and/or save report
s.show_report()
s.save_report('/Users/Ben/Desktop/report.pdf')

In [27]:
# Save output image
imsave("/Users/Ben/Desktop/testimage.tif",s.average_image.astype('float32'))